In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../'))
from utility.print_summary import print_summary

In [2]:
import pandas as pd
import numpy as np


df = pd.read_csv('/Users/dhruvpatel/Desktop/projects/DealPredection/data/vehicles.csv')

print(df.head())

           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       NaN  ...  NaN   NaN

# standardizing 

In [3]:
#  standardization  and extracting info from model and description.
from DataCleaning.data_model import process_car_dataset

featured_eng=process_car_dataset(df)


Starting data extraction and cleaning...


/Users/dhruvpatel/miniconda3/envs/Deal_Predection/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data extraction and cleaning completed!


In [4]:
# dropping  unnecessory columns
from DataCleaning.data_cleaning import drop_unnecessary_columns , drop_rows_with_few_missing_values

df, summary = drop_unnecessary_columns(df) 
print_summary(summary)


Summary
Original Columns: 26
Final Columns: 16
Dropped Columns: ['url', 'image_url', 'county', 'VIN', 'size', 'condition', 'posting_date', 'cylinders', 'region', 'region_url']
Missing Columns: []
Columns Dropped Count: 10
Columns Remaining: ['id', 'price', 'year', 'manufacturer', 'model', 'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'type', 'paint_color', 'description', 'state', 'lat', 'long']


In [5]:
# dropping rows due to high NAs 

df, summary = df_cleaned, summary = drop_rows_with_few_missing_values(df) 

print_summary(summary)

Summary
Original Rows: 426,880
Final Rows: 390,217
Dropped Rows: 36,663
Drop Percentage: 8.59
Columns With Few Missing: ['year', 'description', 'fuel', 'odometer', 'lat', 'long', 'transmission', 'model', 'manufacturer']
Missing Columns: []
Missing Counts Before: {'year': np.int64(1205), 'description': np.int64(70), 'fuel': np.int64(3013), 'odometer': np.int64(4400), 'lat': np.int64(6549), 'long': np.int64(6549), 'transmission': np.int64(2556), 'model': np.int64(5277), 'manufacturer': np.int64(17646)}
Total Missing Values: 47265


# Filling missing values 

### Title_status

In [6]:
# filling missing values in title status with 'missing' 
from DataCleaning.data_title_status import fill_missing_values 

df,summary = fill_missing_values(df) 
print_summary(summary)

Summary
Column Name: title_status
Fill Value: missing
Missing Before: 6730
Missing After: 0
Values Filled: 6730
Total Rows: 390,217


###  Transmission

In [7]:
from DataCleaning.data_transmission import fill_missing_values_transmission, convert_transmission_to_automatic 


df,summary = fill_missing_values_transmission(df) 
print_summary(summary)
print_summary(summary)

Summary
Column Name: transmission
Fill Value: automatic
Missing Before: 0
Missing After: 0
Values Filled: 0
Total Rows: 390,217
Summary
Column Name: transmission
Fill Value: automatic
Missing Before: 0
Missing After: 0
Values Filled: 0
Total Rows: 390,217


In [8]:
df, summary = convert_transmission_to_automatic(df)

print_summary(summary)

Summary
Total Rows: 390,217
Original Unique Values: 3
New Unique Values: 2
Original Value Counts: {'automatic': 309758, 'other': 59435, 'manual': 21024}
New Value Counts: {'automatic': 369193, 'manual': 21024}
Converted To Automatic: 369,193


# drive column cleaning 

In [9]:
# standardasition 
from DataCleaning.data_drive import clean_drive_column
df,summary=clean_drive_column(df, 'drive')  


In [10]:
# filling null values from research : (na values)
from DataCleaning.data_drive import fill_missing_drive_from_reference


df,summary = fill_missing_drive_from_reference(df,
                                       reference_file='/Users/dhruvpatel/Desktop/projects/DealPredection/data/models_with_drive.csv'
                                                 )

print_summary(summary)

Summary
Total Rows: 390,217
Missing Before: 117577
Missing After: 100574
Values Filled: 17,003
Models Not Found: 100,574
Mappings Loaded: 83


# Model 

Clean in stages:
* Stage 1: Remove obvious junk (numbers, too short, too long)
* Stage 2: Extract core model from complex strings
* Stage 3: Standardize spelling and format
* Stage 4: Apply manufacturer-specific rules

In [11]:
# Stage 1 : Remove obvious junk (only numbers, too short, too long)

from DataCleaning.data_model import remove_numerical_models

df,summary =remove_numerical_models(df)

print_summary(summary)

Debug: DataFrame shape after cleaning: (377123, 16)
Step: remove_numerical_models
Rows removed due to numerical only: 11499
Rows removed due to length > 40: 1595
Total rows removed: 13094
Summary
Total Rows Before: 390,217
Total Rows After: 377,123
Rows Removed: 13,094
Numerical Removed: 11499
Length Removed: 1595


In [12]:
df['manufacturer']

27              gmc
28        chevrolet
29        chevrolet
30           toyota
31             ford
            ...    
426875       nissan
426876        volvo
426877     cadillac
426878        lexus
426879          bmw
Name: manufacturer, Length: 377123, dtype: object

In [13]:
from DataCleaning.data_model import clean_models_with_list_optimized
 
df,summary = clean_models_with_list_optimized(df)  

print_summary(summary)

Starting optimized model cleaning...
Creating optimized lookup tables...
Processing unique model values...


Processing unique models: 100%|██████████| 22159/22159 [04:34<00:00, 80.63it/s] 


Applying results to DataFrame...
Found 358210 models to update.
Step: clean_models_with_list_optimized
Total rows modified: 263172

Sample transformations:
  'sierra 1500 crew cab slt' (gmc) -> 'Sierra' (gmc)
  'silverado 1500' (chevrolet) -> 'silverado' (chevrolet)
  'silverado 1500 crew' (chevrolet) -> 'silverado' (chevrolet)
  'tundra double cab sr' (toyota) -> 'Tundra' (toyota)
  'f-150 xlt' (ford) -> 'F-150' (ford)
  'sierra 2500 hd extended cab' (gmc) -> 'Sierra' (gmc)
  'silverado 1500 double' (chevrolet) -> 'silverado' (chevrolet)
  'colorado extended cab' (chevrolet) -> 'colorado' (chevrolet)
  'corvette grand sport' (chevrolet) -> 'corvette' (chevrolet)
  'cherokee' (jeep) -> 'Cherokee' (jeep)
Summary
Total Rows: 377,123
Rows Modified: 263172
Models Updated: 278506
Manufacturers Updated: 65490
Unique Models Processed: 22,159
Lookup Tables Created: 1,923


In [14]:
df.isnull().sum()

id                   0
price                0
year                 0
manufacturer         0
model                0
fuel                 0
odometer             0
title_status         0
transmission         0
drive            97898
type             79372
paint_color     113544
description          0
state                0
lat                  0
long                 0
dtype: int64

In [15]:
# clean model 
from DataCleaning.data_model import filter_by_value_counts

df = filter_by_value_counts(df, 'model', min_count=10) 

# Type

In [16]:
from DataCleaning.data_type import drop_na_drive_type  

df, summary = drop_na_drive_type(df)
print_summary(summary)

Summary
Original Rows: 370,883
Final Rows: 331,111
Rows Dropped: 39772
Both Missing: 39772


In [17]:
# cleaning and standardasition  (type) 
from DataCleaning.data_type import replace_values 
df, summary =replace_values(df, 'type', {'mini van': 'minivan', 'mini-van': 'minivan'})
print_summary(summary)


Summary
Total Rows: 331,111
Rows Changed: 4415
Replacements: {'mini van': 'minivan', 'mini-van': 'minivan'}


In [18]:
# filling null values based on data present. 
# First, let's see what we're working with

from DataCleaning.data_type import fill_type_from_model

df, summary = fill_type_from_model(df)

print_summary(summary)

Summary
Total Rows: 331,111
Missing Before: 37782
Missing After: 91
Values Filled: 37691
Mapping Created: 972


In [19]:
# finally drop type nulls 
from DataCleaning.data_type import drop_na_type

df_clean, summary =drop_na_type(df)

print_summary(summary)

Summary
Original Rows: 331,111
Final Rows: 331,020
Rows Dropped: 91
Missing Values: 91


# Drive

In [20]:
# impute drive 1 ( based on cross tab type )

from DataCleaning.data_drive import impute_drive_from_type

df, summary = impute_drive_from_type(df)
print_summary(summary)


Summary
Total Rows: 331,111
Missing Before: 56145
Missing After: 0
Values Imputed: 56145


### Manufacturer 

In [21]:
from DataCleaning.data_manufacturers import standardize_manufacturer 

df, summary =standardize_manufacturer(df)
print_summary(summary)

Summary
Total Rows: 331,111
Rows Changed: 1094
Replacements: {'land rover': 'land-rover', 'rover': 'land-rover'}


### Paint Color 

In [22]:
from DataCleaning.data_paint_color import fill_paint_color_nulls 

# Usage
df, summary = fill_paint_color_nulls(df)

print_summary(summary)

Summary
Total Rows: 331,111
Nulls Before: 77782
Nulls After: 0
Filled Count: 77782
Fill Percentage: 100.0
Manufacturer State Combinations: 2,600
Successful Combinations: 2,553


# census_region

In [23]:
from  DataCleaning.data_census_region import add_census_divisions_abbrev , validate_regions

# Usage
df, summary = add_census_divisions_abbrev(df)

print_summary(summary)

Summary
Total Rows: 331,111
Mapped Rows: 331111
Unmapped Rows: 0
Unmapped States: []
Divisions Found: 9


# Price

In [24]:
from DataCleaning.data_price import clean_price_data 
# Usage example:
df, summary = clean_price_data(df, 'price')
print_summary(summary)

Summary
Original Rows: 331,111
Rows After Cleaning: 301,980
Total Rows Dropped: 29,131
Zero Price Dropped: 22,277
Out Of Range Dropped: 6,854
Drop Rate Percent: 8.8
Price Range Applied: $500 - $300,000
Final Price Min: 500
Final Price Max: 290000
Final Price Mean: 19387.46
Final Price Std: 14268.04


# Fuel

In [25]:
from DataCleaning.data_fuel import  convert_fuel_to_gas


df,summary = convert_fuel_to_gas(df)
print_summary(summary)

Summary
Total Rows: 301,980
Original Unique Values: 5
New Unique Values: 4
Original Value Counts: {'gas': 257460, 'other': 24348, 'diesel': 14848, 'hybrid': 3974, 'electric': 1350}
New Value Counts: {'gas': 281808, 'diesel': 14848, 'hybrid': 3974, 'electric': 1350}
Converted To Gas: 24,348


# odometer 

In [26]:
# Import the functions
from DataCleaning.data_odometer import process_odometer_column



# Clean the data (recommended approach)
df, summary = process_odometer_column(df, 'odometer')

print_summary(summary)

Step 1: Removing extreme odometer values...

Step 2: Validating odometer values...

Final Results:
Original rows: 301,980
Final rows: 301,586
Total removed: 394 (0.13%)
Final odometer range: 0 - 500,000
Final odometer mean: 89,618
Summary
Original Rows: 301,980
Final Rows: 301,586
Total Removed: 394
Total Removal Percentage: 0.13
Removal Summary: {'original_rows': 301980, 'cleaned_rows': 301586, 'removed_rows': 394, 'removal_percentage': 0.13, 'min_threshold': 0, 'max_threshold': 500000, 'extreme_low_count': 0, 'extreme_high_count': 394, 'null_count': np.int64(0), 'original_stats': {'count': 301980.0, 'mean': 92539.45117557455, 'std': 147118.78384392778, 'min': 0.0, '25%': 35326.0, '50%': 83301.0, '75%': 132000.0, 'max': 10000000.0}, 'cleaned_stats': {'count': 301586.0, 'mean': 89618.33646787317, 'std': 62483.48819244328, 'min': 0.0, '25%': 35290.0, '50%': 83141.5, '75%': 131904.0, 'max': 500000.0}}
Validation Summary: {'total_rows': 301586, 'valid_rows': 301586, 'dropped_rows': 0, 'dr


# validation columns 

1. census_region ✅ 
2. drive ✅
3. fuel ✅
4. lat  
5. long
6. manufacturer ✅ 
7. model ✅ 
8.  ✅
9. paint_color ✅
10. price ✅
11. state ✅
12. title_status ✅
13. transmission ✅
14. type ✅
15. year ✅

In [27]:
# 1 census_region 
df, summary = validate_regions(df) 

print_summary(summary)

Summary
Original Rows: 301,586
Rows After Filtering: 301,586
Rows Dropped: 0
Invalid Regions Found: []
Drop Rate Percent: 0.0
Valid Regions: ['Mountain', 'West South Central', 'South Atlantic', 'West North Central', 'East North Central', 'Middle Atlantic', 'Pacific', 'East South Central', 'New England']


In [28]:
# 2 
from DataCleaning.data_year import validate_years
# Usage
df, summary = validate_years(df, year_column='year', min_year=1990)

print_summary(summary)

Summary
Original Rows: 301,586
Filtered Rows: 295,997
Dropped Rows: 5,589
Drop Percentage: 1.85
Min Year Threshold: 1,990
Original Year Range: 1900.0 - 2022.0
New Year Range: 1990.0 - 2022.0
Null Years: 0
Invalid Years Count: 69
Invalid Years Breakdown: {1900.0: np.int64(1), 1905.0: np.int64(1), 1915.0: np.int64(1), 1918.0: np.int64(1), 1922.0: np.int64(1), 1923.0: np.int64(20), 1924.0: np.int64(6), 1925.0: np.int64(4), 1926.0: np.int64(3), 1927.0: np.int64(14), 1928.0: np.int64(20), 1929.0: np.int64(25), 1930.0: np.int64(27), 1931.0: np.int64(15), 1932.0: np.int64(21), 1933.0: np.int64(7), 1934.0: np.int64(14), 1935.0: np.int64(8), 1936.0: np.int64(9), 1937.0: np.int64(12), 1938.0: np.int64(7), 1939.0: np.int64(11), 1940.0: np.int64(23), 1941.0: np.int64(17), 1942.0: np.int64(4), 1946.0: np.int64(21), 1947.0: np.int64(17), 1948.0: np.int64(14), 1949.0: np.int64(11), 1950.0: np.int64(20), 1951.0: np.int64(30), 1952.0: np.int64(27), 1953.0: np.int64(28), 1954.0: np.int64(16), 1955.0: np

In [29]:
# 3
# transmission column 

from DataCleaning.data_transmission import  validate_transmission_values 

df, validation_summary = validate_transmission_values(df)

print_summary(summary)

Summary
Original Rows: 301,586
Filtered Rows: 295,997
Dropped Rows: 5,589
Drop Percentage: 1.85
Min Year Threshold: 1,990
Original Year Range: 1900.0 - 2022.0
New Year Range: 1990.0 - 2022.0
Null Years: 0
Invalid Years Count: 69
Invalid Years Breakdown: {1900.0: np.int64(1), 1905.0: np.int64(1), 1915.0: np.int64(1), 1918.0: np.int64(1), 1922.0: np.int64(1), 1923.0: np.int64(20), 1924.0: np.int64(6), 1925.0: np.int64(4), 1926.0: np.int64(3), 1927.0: np.int64(14), 1928.0: np.int64(20), 1929.0: np.int64(25), 1930.0: np.int64(27), 1931.0: np.int64(15), 1932.0: np.int64(21), 1933.0: np.int64(7), 1934.0: np.int64(14), 1935.0: np.int64(8), 1936.0: np.int64(9), 1937.0: np.int64(12), 1938.0: np.int64(7), 1939.0: np.int64(11), 1940.0: np.int64(23), 1941.0: np.int64(17), 1942.0: np.int64(4), 1946.0: np.int64(21), 1947.0: np.int64(17), 1948.0: np.int64(14), 1949.0: np.int64(11), 1950.0: np.int64(20), 1951.0: np.int64(30), 1952.0: np.int64(27), 1953.0: np.int64(28), 1954.0: np.int64(16), 1955.0: np

In [30]:
# 4 
from DataCleaning.data_fuel import validate_fuel_values
df , summary = validate_fuel_values(df) 
print_summary(summary)

Summary
Total Rows: 295,997
Valid Rows: 295,997
Dropped Rows: 0
Drop Percentage: 0.0
Valid Values: ['gas', 'diesel', 'hybrid', 'electric']
Original Value Counts: {'gas': 275998, 'diesel': 14678, 'hybrid': 3972, 'electric': 1349}
Invalid Values: {}
Null Values: 0
Validation Passed: True


In [31]:
# 5 
from DataCleaning.data_title_status import validate_title_status_values 

df, summary = validate_title_status_values(df)

print_summary(summary)

Summary
Total Rows: 295,997
Valid Rows: 295,997
Dropped Rows: 0
Drop Percentage: 0.0
Valid Values: ['clean', 'rebuilt', 'missing', 'salvage', 'lien', 'parts only']
Original Value Counts: {'clean': 280797, 'missing': 5795, 'rebuilt': 5514, 'salvage': 2753, 'lien': 1076, 'parts only': 62}
Invalid Values: {}
Null Values: 0
Validation Passed: True


In [32]:
# 6 
from DataCleaning.data_type import validate_type_values
df, summary = validate_type_values(df, standardize_case=True)

print_summary(summary)

Summary
Total Rows: 295,997
Valid Rows: 295,921
Dropped Rows: 76
Drop Percentage: 0.03
Valid Values: ['sedan', 'suv', 'pickup', 'truck', 'other', 'coupe', 'hatchback', 'wagon', 'van', 'convertible', 'minivan', 'bus', 'offroad']
Original Value Counts: {'sedan': 82353, 'SUV': 71130, 'pickup': 39748, 'truck': 25476, 'other': 17239, 'coupe': 15452, 'hatchback': 14747, 'wagon': 9678, 'van': 8135, 'minivan': 5657, 'convertible': 5641, 'offroad': 409, 'bus': 256}
Invalid Values: {}
Null Values: 76
Validation Passed: False
Case Standardized: True


In [33]:
# 7
# Usage

from DataCleaning.data_manufacturers import  validate_manufacturers 
df, summary = validate_manufacturers(df)
print_summary(summary)

Summary
Original Rows: 295,921
Filtered Rows: 294,000
Dropped Rows: 1,921
Drop Percentage: 0.65
Valid Manufacturers Count: 72
Found Manufacturers Count: 61
Invalid Manufacturers: {'crosley': np.int64(1215), 'austin': np.int64(657), 'harley-davidson': np.int64(38), 'harley': np.int64(6), 'desoto': np.int64(5)}


In [34]:
# 8 
from DataCleaning.data_paint_color import validate_paint_color
df, summary = validate_paint_color(df)

print_summary(summary)

Summary
Original Rows: 294,000
Final Rows: 294,000
Rows Dropped: 0
Invalid Values: 0
Valid Colors: ['white', 'black', 'silver', 'blue', 'red', 'grey', 'green', 'brown', 'custom', 'orange', 'yellow', 'purple']


In [35]:
# 9 
from DataCleaning.data_state import validate_state
df, summary = validate_state(df)

print_summary(summary)

Summary
Original Rows: 294,000
Final Rows: 294,000
Rows Dropped: 0
Invalid Values: 0
Valid States: ['ca', 'fl', 'tx', 'ny', 'oh', 'mi', 'pa', 'or', 'wa', 'nc', 'wi', 'tn', 'co', 'il', 'id', 'va', 'nj', 'az', 'ma', 'mn', 'ia', 'ga', 'ks', 'mt', 'in', 'ok', 'sc', 'ct', 'md', 'al', 'ky', 'mo', 'ak', 'nm', 'nv', 'ar', 'dc', 'nh', 'la', 'me', 'vt', 'hi', 'ri', 'ut', 'sd', 'wv', 'ms', 'ne', 'de', 'wy', 'nd']


In [36]:
# 10 model 
from DataCleaning.data_model import validate_model_frequency

df_clean, summary = validate_model_frequency(df, min_count=10)
print_summary(summary)

Summary
Original Rows: 294,000
Final Rows: 293,355
Rows Dropped: 645
Min Count Threshold: 10
Models Kept: 808
Models Dropped: 112
Infrequent Model Rows: 645


In [37]:
# 11 Drive  
from DataCleaning.data_drive import validate_drive_values
df, summary = validate_drive_values(df)
print_summary(summary)

Summary
Original Rows: 294,000
Final Rows: 294,000
Rows Dropped: 0
Invalid Values: 0
Valid Drives: ['4wd', 'fwd', 'rwd']
Value Counts: {'4wd': 133495, 'fwd': 114880, 'rwd': 45625}


In [38]:
# 13
from DataCleaning.data_odometer import validate_odometer 

df, summary = validate_odometer(df, min_miles=0, max_miles=500000)
print_summary(summary)

Summary
Original Rows: 294,000
Final Rows: 294,000
Rows Dropped: 0
Invalid Values: 0
Min Miles: 0
Max Miles: 500,000


In [39]:
from DataCleaning.data_lat_long import validate_usa_coordinates
df, summary = validate_usa_coordinates(df)
print_summary(summary)

Summary
Original Rows: 294,000
Final Rows: 293,943
Rows Dropped: 57
Invalid Coordinates: 57
Lat Bounds: 18.0 - 72.0
Long Bounds: -180.0 - -66.0


In [41]:
df.isna().sum()

id               0
price            0
year             0
manufacturer     0
model            0
fuel             0
odometer         0
title_status     0
transmission     0
drive            0
type             0
paint_color      0
description      0
state            0
lat              0
long             0
census_region    0
dtype: int64